In [1]:
import evaluate
chrf = evaluate.load("chrf")


In [2]:
file1=open('Flores/eng_Latn.dev','r') # string of english sentences 
file2=open('Flores/grn_Latn.dev','r') # string of guarani counterparts


ENG=file1.read()
GRN=file2.read()

In [3]:
ENGs=ENG.splitlines() #sentence by sentence list of the english data
GRNs=GRN.splitlines() #sentence by sentence list of the guarani data

In [4]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_dir = "mbart50-finetuned-gn/checkpoint-1248"

# Load the tokenizer
tokenizer = MBart50TokenizerFast.from_pretrained(model_dir)

# Load the model
model = MBartForConditionalGeneration.from_pretrained(model_dir)


In [5]:
import transformers
print(transformers.__version__)

4.51.3


In [6]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_dir = "mbart50-finetuned-gn/checkpoint-625"

# Load tokenizer and model
tokenizer = MBart50TokenizerFast.from_pretrained(model_dir)
model = MBartForConditionalGeneration.from_pretrained(model_dir)

# Add missing lang code if needed
if "gn_XX" not in tokenizer.lang_code_to_id:
    tokenizer.add_special_tokens({'additional_special_tokens': ["gn_XX"]})
    tokenizer.lang_code_to_id["gn_XX"] = len(tokenizer) - 1
    model.resize_token_embeddings(len(tokenizer), mean_resizing=False)

In [8]:
# Translation test
tokenizer.src_lang = "en_XX"
target_lang = "gn_XX"
input_text = "Hello"

inputs = tokenizer(input_text, return_tensors="pt")
translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[target_lang])

translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
print(translated_text)

In [9]:
def translation(source):
    # ensure tokenizer and target are correct
    tokenizer.src_lang = "en_XX"
    target_lang = "gn_XX"

    translation_list=[]
    for sent in source:
        inputs = tokenizer(input_text, return_tensors="pt")
        translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[target_lang])
        translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
        translation_list.append(translated_text)
    return translation_list
    

In [10]:
predictions=translation(ENGs)
references=[[sent] for sent in GRNs]

In [11]:
results = chrf.compute(predictions=predictions, references=references, word_order=2)
print(results)

{'score': 0.0, 'char_order': 6, 'word_order': 2, 'beta': 2}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
